# Near matching tests
Near matching as of 2016-07-23 (djb)

## Analytic framework
The witnesses start and end with perfect matches (*abcd* and *efgh*, respectively). Witness **A** has one token in the middle (*0123*) and witness **B** has two (*012x*, *01xx*) or three (*012x*, *01xx*, *0xxx*). The two or three candidates for alignment in witness **B** are all partial matches to the middle token in **A**, with different degrees of similarity. All permutations of the candidates in **B** are tested to determine whether **A** is aligned with the correct one.

## How near matching works
1. Walking from right to left through the ranks, find ranks where not all witnesses are present (*gaps*).
1. At each rank with a gap, walk through the witnesses in alphabetical order.
1. For each witness at a gap, scan left until you hit the first node with a reading for that witness (or the start node). Whatever you hit is the *prior node*.
1. Build a *near match table* of distances between the token at the prior node and each rank from the prior node rank through the rank with the gap being processed. 
1. Verify that there is a *need* to move: the match at the current location of the token (the prior node) has to be lower than the match at some other rank in the near match table.
1. Verify that it is *possible* to move:
    1. The prior node must have tokens (it can't be the start node).
    1. The prior node cannot contain any witness already present in any node at the current rank, since we can't wind up with two tokens for the same witness at a single rank.
1. If moving is both necessary and possible, move the token to the new location. Otherwise leave it where it is.
1. Continue until all gaps in all witnesses at all ranks have been processed.

## Without near matching, candidate always stays left, even if right is closer
Not the desired output: Here *0123* in **A** is closer to *012x* (right) than to *01xx* (left), but it stays left anyway.

In [198]:
%reload_ext autoreload
%autoreload 2
from collatex import *
collation = Collation()
collation.add_plain_witness("A", "abcd 0123 efgh")
collation.add_plain_witness("B", "abcd 01xx 012x efgh")
alignment_table = collate(collation, segmentation=False)
print(alignment_table)

+---+------+------+------+------+
| A | abcd | 0123 | -    | efgh |
| B | abcd | 01xx | 012x | efgh |
+---+------+------+------+------+


## With near matching and two choices, candidate is aligned correctly

In the example below, *0123* in **A** is closer to *012x* (left) in **B**, and it correctly stays left.

In [199]:
# Two candidates
# Closer match is left, no movement
collation = Collation()
collation.add_plain_witness("A", "abcd 0123 efgh")
collation.add_plain_witness("B", "abcd 012x 01xx efgh")
alignment_table = collate(collation, near_match=True, segmentation=False)
print(alignment_table)

+---+------+------+------+------+
| A | abcd | 0123 | -    | efgh |
| B | abcd | 012x | 01xx | efgh |
+---+------+------+------+------+


In the example below, *0123* in **A** is closer to *012x* (right) in **B**, and it correctly moves right.

In [200]:
# Two candidates
# Same input as above, but closer match is right, so moves
collation = Collation()
collation.add_plain_witness("A", "abcd 0123 efgh")
collation.add_plain_witness("B", "abcd 01xx 012x efgh")
alignment_table = collate(collation, near_match=True, segmentation=False)
print(alignment_table)

+---+------+------+------+------+
| A | abcd | -    | 0123 | efgh |
| B | abcd | 01xx | 012x | efgh |
+---+------+------+------+------+


## With near matching and three or more choices, the alignment is correct regardless

### If the closest match is left, the candidate correctly always stays left

In [201]:
# Three candidates, closest is left, match rank 0 1 2 (0 is closest)
collation = Collation()
collation.add_plain_witness("A", "abcd 0123 efgh")
collation.add_plain_witness("B", "abcd 012x 01xx 0xxx efgh")
alignment_table = collate(collation, near_match=True, segmentation=False)
print(alignment_table)

+---+------+------+------+------+------+
| A | abcd | 0123 | -    | -    | efgh |
| B | abcd | 012x | 01xx | 0xxx | efgh |
+---+------+------+------+------+------+


In [202]:
# Three candidates, closest is left, match rank 0 2 1 (0 is closest)
collation = Collation()
collation.add_plain_witness("A", "abcd 0123 efgh")
collation.add_plain_witness("B", "abcd 012x 0xxx 01xx efgh")
alignment_table = collate(collation, near_match=True, segmentation=False)
print(alignment_table)

+---+------+------+------+------+------+
| A | abcd | 0123 | -    | -    | efgh |
| B | abcd | 012x | 0xxx | 01xx | efgh |
+---+------+------+------+------+------+


### If the closest match is right, the candidate correctly always moves right

In [203]:
# Three candidates, closest is right, match rank 1 2 0 (0 is closest)
collation = Collation()
collation.add_plain_witness("A", "abcd 0123 efgh")
collation.add_plain_witness("B", "abcd 01xx 0xxx 012x efgh")
alignment_table = collate(collation, near_match=True, segmentation=False)
print(alignment_table)

+---+------+------+------+------+------+
| A | abcd | -    | -    | 0123 | efgh |
| B | abcd | 01xx | 0xxx | 012x | efgh |
+---+------+------+------+------+------+


In [204]:
# Three candidates, closest is right, match rank 2 1 0 (0 is closest)
collation = Collation()
collation.add_plain_witness("A", "abcd 0123 efgh")
collation.add_plain_witness("B", "abcd 0xxx 01xx 012x efgh")
alignment_table = collate(collation, near_match=True, segmentation=False)
print(alignment_table)

+---+------+------+------+------+------+
| A | abcd | -    | -    | 0123 | efgh |
| B | abcd | 0xxx | 01xx | 012x | efgh |
+---+------+------+------+------+------+


### If the closest match is in the middle, the always correctly moves to the middle

In [205]:
# Three candidates, closest is middle, match rank 1 0 2 (0 is closest)
collation = Collation()
collation.add_plain_witness("A", "abcd 0123 efgh")
collation.add_plain_witness("B", "abcd 01xx 012x 0xxx efgh")
alignment_table = collate(collation, near_match=True, segmentation=False)
print(alignment_table)

+---+------+------+------+------+------+
| A | abcd | -    | 0123 | -    | efgh |
| B | abcd | 01xx | 012x | 0xxx | efgh |
+---+------+------+------+------+------+


In [206]:
# Three candidates, closest is middle, match rank 2 0 1 (0 is closest)
collation = Collation()
collation.add_plain_witness("A", "abcd 0123 efgh")
collation.add_plain_witness("B", "abcd 0xxx 012x 01xx efgh")
alignment_table = collate(collation, near_match=True, segmentation=False)
print(alignment_table)

+---+------+------+------+------+------+
| A | abcd | -    | 0123 | -    | efgh |
| B | abcd | 0xxx | 012x | 01xx | efgh |
+---+------+------+------+------+------+


## Three witnesses, two of which have gaps
We expect:

    +---+------+--------+--------+--------+--------+--------+------+
    | A | abcd | -      | -      | 012345 | -      |        | efgh |
    | B | abcd | 0xxxxx | 01xxxx | 01234x | 012xxx | 0123xx | efgh |
    | C | abcd | -      | 01xxxx | -      | -      | zz23xx | efgh |
    +---+------+--------+--------+--------+--------+--------+------+

In [207]:
%reload_ext autoreload
%autoreload 2
from collatex import *
collation = Collation()
collation.add_plain_witness("A", "abcd 012345 efgh")
collation.add_plain_witness("B", "abcd 0xxxxx 01xxxx 01234x 012xxx 0123xx efgh")
collation.add_plain_witness("C", "abcd 01xxxx zz23xx efgh")
exact = collate(collation, segmentation=False, near_match=False)
print('Without near matching:')
print(exact)
print('Target output:')
print("""\
+---+------+--------+--------+--------+--------+--------+------+
| A | abcd | -      | -      | 012345 | -      | -      | efgh |
| B | abcd | 0xxxxx | 01xxxx | 01234x | 012xxx | 0123xx | efgh |
| C | abcd | -      | 01xxxx | -      | -      | zz23xx | efgh |
+---+------+--------+--------+--------+--------+--------+------+""")
near = collate(collation, segmentation=False, near_match=True)
print('With near matching:')
print(near)

Without near matching:
+---+------+--------+--------+--------+--------+--------+------+
| A | abcd | 012345 | -      | -      | -      | -      | efgh |
| B | abcd | 0xxxxx | 01xxxx | 01234x | 012xxx | 0123xx | efgh |
| C | abcd | -      | 01xxxx | zz23xx | -      | -      | efgh |
+---+------+--------+--------+--------+--------+--------+------+
Target output:
+---+------+--------+--------+--------+--------+--------+------+
| A | abcd | -      | -      | 012345 | -      | -      | efgh |
| B | abcd | 0xxxxx | 01xxxx | 01234x | 012xxx | 0123xx | efgh |
| C | abcd | -      | 01xxxx | -      | -      | zz23xx | efgh |
+---+------+--------+--------+--------+--------+--------+------+
With near matching:
+---+------+--------+--------+--------+--------+--------+------+
| A | abcd | -      | -      | 012345 | -      | -      | efgh |
| B | abcd | 0xxxxx | 01xxxx | 01234x | 012xxx | 0123xx | efgh |
| C | abcd | -      | 01xxxx | -      | -      | zz23xx | efgh |
+---+------+--------+--------+--

In [208]:
%reload_ext autoreload
%autoreload 2
from collatex import *
collation = Collation()
collation.add_plain_witness("A", "abcd 01234x 0123xx efgh")
collation.add_plain_witness("B", "abcd 01xxxx 01234x 01234x 012xxx 0123xx efgh")
collation.add_plain_witness("C", "abcd 012345 efgh")
exact_table = collate(collation, segmentation=False, near_match=False)
print('Without near matching:')
print(exact_table)
print('Target output:')
print("""\
+---+------+--------+--------+--------+--------+--------+------+
| A | abcd | -      | 01234x | -      | -      | 0123xx | efgh |
| B | abcd | 01xxxx | 01234x | 01234x | 012xxx | 0123xx | efgh |
| C | abcd | -      | 012345 | -      | -      | -      | efgh |
+---+------+--------+--------+--------+--------+--------+------+""")
near_table = collate(collation, segmentation=False, near_match=True)
print(near_table)

Without near matching:
+---+------+--------+--------+--------+--------+--------+------+
| A | abcd | -      | 01234x | -      | -      | 0123xx | efgh |
| B | abcd | 01xxxx | 01234x | 01234x | 012xxx | 0123xx | efgh |
| C | abcd | 012345 | -      | -      | -      | -      | efgh |
+---+------+--------+--------+--------+--------+--------+------+
Target output:
+---+------+--------+--------+--------+--------+--------+------+
| A | abcd | -      | 01234x | -      | -      | 0123xx | efgh |
| B | abcd | 01xxxx | 01234x | 01234x | 012xxx | 0123xx | efgh |
| C | abcd | -      | 012345 | -      | -      | -      | efgh |
+---+------+--------+--------+--------+--------+--------+------+
+---+------+--------+--------+--------+--------+--------+------+
| A | abcd | -      | 01234x | -      | -      | 0123xx | efgh |
| B | abcd | 01xxxx | 01234x | 01234x | 012xxx | 0123xx | efgh |
| C | abcd | -      | 012345 | -      | -      | -      | efgh |
+---+------+--------+--------+--------+--------+----